# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import gmaps.datasets

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
weather_data = pd.read_csv("WeatherPy.csv")
weather_data

,City,CityID,Country,Lat,Long,temp,humidity,cloudiness,wind speed
0,port alfred,964432,ZA,-33.59,26.89,66.99,91,69,1.83
1,santa marta,3668605,CO,11.24,-74.20,79.83,71,3,16.40
2,hope,4115181,US,33.67,-93.59,50.36,81,90,5.82
3,rikitea,4030556,PF,-23.12,-134.97,77.14,67,0,5.32
4,provideniya,4031574,RU,64.38,-173.30,23.83,96,100,20.76
...,...,...,...,...,...,...,...,...,...
565,moses lake,5803990,US,47.13,-119.28,58.37,33,1,11.41
566,bilma,2446796,NE,18.69,12.92,83.01,12,0,8.46
567,oum hadjer,2426370,TD,13.30,19.70,83.97,13,8,16.78
568,iwanai,2129868,JP,42.97,140.51,53.56,58,93,8.28


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
idealtemp = (weather_data['temp']<80) & (weather_data['temp']>50)
idealhumid = weather_data['humidity']<40
idealcloud = weather_data['cloudiness']<20
idealwind = weather_data['wind speed']<10

hotel_df = weather_data[idealtemp & idealhumid & idealcloud & idealwind]
hotel_df

,City,CityID,Country,Lat,Long,temp,humidity,cloudiness,wind speed
156,kidal,2455290,ML,18.44,1.41,79.74,15,16,5.57
165,san juan,3837213,AR,-31.54,-68.54,69.01,39,1,8.86
169,medina,109223,SA,24.47,39.61,77.00,15,13,3.49
192,calvinia,3369174,ZA,-31.47,19.78,64.40,21,2,8.05
250,balkhash,1525798,KZ,46.84,74.98,66.20,22,0,8.95
310,goure,2444219,NE,13.98,10.27,76.37,14,8,7.23
404,tanout,2439155,NE,14.97,8.89,76.57,13,0,7.11
420,chardara,1524889,KZ,41.25,67.97,71.31,28,0,3.31
467,xining,1788852,CN,36.62,101.77,66.18,10,4,5.44


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
hotel_df['Hotel Name'] = ""
hotel_df

C:\Users\sayak\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City,CityID,Country,Lat,Long,temp,humidity,cloudiness,wind speed,Hotel Name
156,kidal,2455290,ML,18.44,1.41,79.74,15,16,5.57,
165,san juan,3837213,AR,-31.54,-68.54,69.01,39,1,8.86,
169,medina,109223,SA,24.47,39.61,77.00,15,13,3.49,
192,calvinia,3369174,ZA,-31.47,19.78,64.40,21,2,8.05,
250,balkhash,1525798,KZ,46.84,74.98,66.20,22,0,8.95,
310,goure,2444219,NE,13.98,10.27,76.37,14,8,7.23,
404,tanout,2439155,NE,14.97,8.89,76.57,13,0,7.11,
420,chardara,1524889,KZ,41.25,67.97,71.31,28,0,3.31,
467,xining,1788852,CN,36.62,101.77,66.18,10,4,5.44,


In [18]:
# Dependencies
import requests
import json

from pprint import pprint

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

def get_nearby_hotel_name(lat, lon):
    params = {
        "location": str(lat) + ',' + str(lon),
        #"rankby":"distance",
        "radius":"5000",
        "type":"hotel",
        "key": g_key
    }
    response = requests.get(base_url, params = params).json()
    
    try:
        return response['results'][1]['name']
    except:
        return 'No hotel name'
   

In [19]:
for i, row in hotel_df.iterrows():
    lat = row['Lat']
    lon = row['Long']
    hotel_name = get_nearby_hotel_name(lat, lon)
    hotel_df.loc[i, 'Hotel Name'] = hotel_name

hotel_df

,City,CityID,Country,Lat,Long,temp,humidity,cloudiness,wind speed,Hotel Name
156,kidal,2455290,ML,18.44,1.41,79.74,15,16,5.57,Marche
165,san juan,3837213,AR,-31.54,-68.54,69.01,39,1,8.86,Hotel Provincial
169,medina,109223,SA,24.47,39.61,77.00,15,13,3.49,Elaf Taiba Hotel فندق إيلاف طيبة
192,calvinia,3369174,ZA,-31.47,19.78,64.40,21,2,8.05,Die Lemoenhuis
250,balkhash,1525798,KZ,46.84,74.98,66.20,22,0,8.95,Hostel 74/76
310,goure,2444219,NE,13.98,10.27,76.37,14,8,7.23,Ces/Mmt Gouré
404,tanout,2439155,NE,14.97,8.89,76.57,13,0,7.11,Tanout Karin Boka
420,chardara,1524889,KZ,41.25,67.97,71.31,28,0,3.31,Apparat Akima Shardarinskogo Rayona
467,xining,1788852,CN,36.62,101.77,66.18,10,4,5.44,青海烟草大厦


In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# # Store the DataFrame Row
# # NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [21]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(weather_data[["Lat","Long"]], weather_data['humidity']))

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))